In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd 
from torch.optim import SGD
from torch.autograd import Variable
from matplotlib.colors import ListedColormap
%matplotlib inline


In [2]:
path = os.getcwd()
data = pd.read_csv(path+"/spambase.csv")
def get_set(data,i):
    N,L = data.shape
    data = data.sample(frac=1).reset_index(drop=True)
    x_train = data.iloc[:int(N*i),:-1].values.reshape(-1,L-1)
    y_train = data.iloc[:int(N*i),-1].values.reshape(-1,1)
    x_test = data.iloc[int(N*i):,:-1].values.reshape(-1,L-1)
    y_test = data.iloc[int(N*i):,-1].values.reshape(-1,1)
    return torch.from_numpy(x_train).float(),torch.from_numpy(x_test).float(),torch.from_numpy(y_train).float(),torch.from_numpy(y_test).float()
train_x,test_x,train_y,test_y = get_set(data,0.7)

torch.Size([1381, 1])

In [5]:
class Perceptron(nn.Module):
    def __init__(self):
        super(Perceptron, self).__init__()
        self.layer = nn.Linear(57, 1)
        
    def forward(self, x):
        x = self.layer(x)
        return x

In [6]:
def loss_func(scores, label, type="svm"):
    assert type=="perceptron" or type=="svm", "loss type error"
    if type == "perceptron":
        # 感知机损失函数，label取值集合为{-1, 1}
        loss = -label*scores
    else:
        # SVM损失函数，label取值集合为{-1, 1}
        loss = 1-label*scores
   
    loss[loss<=0] = 0
    return torch.sum(loss)


In [7]:
def sign(x):
    x[x>=0] = 1
    x[x<0] = -1
    return x


In [8]:
# 预测函数
def pred(x):
    return sign(x)


In [9]:
model = Perceptron()
optim_func = SGD(model.parameters(), lr=0.0001)
#for name,param in model.named_parameters():
#   print(name, param)

for epoch in range(15):
    inputs, targets = train_x, train_y
    label = train_y
    # 前向传播
    scores = model(inputs).squeeze(1)
    loss = loss_func(scores, label, "svm")
    # 反向传播
    optim_func.zero_grad()
    loss.backward()
    optim_func.step()


    if epoch % 1000 == 0:
        # 计算分类的准确率
        inputs, targets = train_x,train_y 
        inputs = Variable(inputs, requires_grad=True)
        label = Variable(targets.int(), requires_grad=False)
        scores = model(inputs).squeeze(-1)
        num_correct = (pred(scores) == label).sum().item()
        acc = num_correct*1.0 / inputs.shape[0]
        print("loss=",loss.detach().numpy(),"acc=", acc)

loss= 9239258.0 acc= 1300.0


tensor([False, False, False,  ..., False, False, False])

In [11]:
for name,param in model.named_parameters():
    print(name, param)


layer.weight Parameter containing:
tensor([[ 41.5677,  81.9470, 115.7110,  26.3568, 124.5816,  39.3264,  49.8463,
          43.6552,  37.7830,  93.7888,  26.0294, 226.2275,  38.3480,  22.0898,
          20.6568,  93.7664,  59.3373,  76.4031, 674.4976,  38.3449, 334.6377,
          11.7455,  40.5993,  35.9823, 204.8103, 104.4862,  91.0173,  48.4175,
          39.0464,  41.3057,  23.8009,  18.4263,  38.5947,  18.8285,  40.7996,
          36.7967,  56.3192,   4.7006,  27.8165,  26.8840,  17.3531,  55.8012,
          19.1381,  31.3781, 115.9066,  62.8950,   2.0793,  12.7161,   9.3216,
          54.9776,   6.9919, 107.6694,  31.6034,  15.5718, -16.2075,  -6.5957,
          -1.6898]], requires_grad=True)
layer.bias Parameter containing:
tensor([395.1791], requires_grad=True)
